<a href="https://colab.research.google.com/github/thabithagogulamanda/FMML-LAB-1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

ANSWER 1

The accuracy of the validation set is a measure of how well the model performs on unseen data. It is affected by the percentage of validation set in different ways, depending on the size and complexity of the dataset and the model.

If we increase the percentage of validation set, we will have more data to evaluate the model, which can lead to a more accurate estimate of the generalization error. However, this also means that we will have less data to train the model, which can reduce its ability to learn from the data and fit the underlying patterns.

ANSWWER 2

The size of the train and validation set can affect how well we can predict the accuracy on the test set using the validation set in different ways. Here are some points to consider:

*The train set is the data that the model learns from, so it should be large enough to capture the complexity and diversity of the real data. A larger train set can help the model generalize better and avoid overfitting, which means that the model performs well on both the train and test set. However, a very large train set can also increase the computational cost and time of training the model, and it may not improve the performance significantly if the data is redundant or noisy.

*The validation set is the data that is used to tune the hyperparameters of the model, such as the learning rate, the number of hidden layers, or the regularization strength. The validation set should be representative of the test set, so that the model can achieve a similar accuracy on both sets. A larger validation set can provide a more reliable estimate of the model’s performance on unseen data, but it also reduces the size of the train set, which may affect the model’s ability to learn from the data.

ANSWER 3

There is no definitive answer to this question, as the optimal percentage for the validation set may depend on various factors, such as the size and distribution of the data, the complexity and architecture of the model, and the number and range of the hyperparameters. However, some general principles are:

The validation set should be large enough to provide a reliable estimate of the model’s performance on unseen data, but not so large that it reduces the amount of data available for training the model. The validation set should be representative of the test set, so that the model can achieve a similar accuracy on both sets. The validation set should be independent of the train and test sets, so that the model does not overfit or underfit to any of them. Based on these principles, some common percentages for the validation.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


ANSWER 1

Yes, averaging the validation accuracy across multiple splits can give more consistent results than using a single split. This is because different splits of the data can have different characteristics and distributions, which can affect the performance of the model. By averaging the validation accuracy across multiple splits, we can reduce the variance and noise in the evaluation metric and get a more reliable estimate of the model’s generalization ability.

One common technique to perform multiple splits of the data is k-fold cross-validation, where the data is divided into k equal-sized subsets, and each subset is used as the validation set once, while the rest are used as the training set. The validation accuracy is then calculated as the average of the k validation scores obtained from each fold. This method can help us avoid overfitting or underfitting the model to a particular split of the data.

ANSWER 2

If you are asking about the web search tool, then it does not give an estimate of test accuracy, but rather it returns relevant results from the web that contain information about test accuracy. You can use these results to learn more about the concept and how to calculate it for different types of tests.

If you are asking about a specific diagnostic test, then the answer may depend on how you define test accuracy. There are different measures of test accuracy, such as sensitivity, specificity, positive predictive value, negative predictive value, and likelihood ratios. Each of these measures has a different interpretation and application. You can read more about them in the first result or the second result.

To compare the accuracy of different tests, you need to consider the pre-test probability of the condition, the prevalence of the condition in the population, and the trade-off between false positives and false negatives. You can also use a confusion matrix to visualize the performance of a test and calculate various metrics. You can find an example of aconfusion matrix in the third result.

ANSWER 3

The number of iterations is an important parameter that affects the accuracy and speed of the estimation process. Generally, the more iterations you perform, the more accurate the estimate will be, but the longer it will take to compute. This is because each iteration adds more information and reduces the uncertainty or error of the estimate. However, there is a trade-off between accuracy and efficiency, and you may not need to perform too many iterations if the estimate is already close enough to the true value or if the improvement is negligible. For example, if you are using Monte Carlo simulations to estimate the value of pi, you can use the formula:

π≈4×number of points inside a circle​/total number of points

The more points you generate, the more accurate your estimate will be, but also the more time and resources you will need. You can see an illustration of this method in this article. Another example is if you are using a solver to find the optimal solution for a mathematical model, such as a neural network or a differential equation. The solver uses an iterative algorithm that updates the solution at each step until it converges to a desired accuracy or tolerance. The smaller the step size or the higher the number of iterations, the more precise the solution will be, but also the slower the convergence will be. You can read more about how to choose the step size and number of iterations for solvers in this documentation.

ANSWER 4

Increasing the iterations may not be a good solution for dealing with a very small train dataset or validation dataset. According to the results, there are two main problems with having a small train dataset or validation dataset:

If the train dataset is too small, the model may not capture enough patterns or generalize well1. This means that the model may have high bias or underfitting, which cannot be solved by increasing the iterations. In fact, increasing the iterations may lead to overfitting, which is another problem that occurs when the model learns too much from the train dataset and fails to perform well on new data. If the validation dataset or test dataset is too small, the performance evaluation may lack statistical significance1. This means that the performance metrics, such as accuracy, precision, recall, etc., may not reflect the true quality of the model. Increasing the iterations will not change the size of the validation dataset or test dataset, so it will not improve the reliability of the performance evaluation. Therefore, increasing the iterations is not a recommended way to deal with a very small train dataset or validation dataset. A better way would be to increase the size of the datasets by collecting more data, using data augmentation techniques, or applying cross-validation methods.